In [1]:
# Описание метода Гаусса

In [2]:
#Описание задания

класс - class Matrix. __str__(self, ); 

1) i := i + lambda * j
2) swap i, j
3) i := i * lambda, lambda != 0

def solve()
    self.forward_gauss()
    1 2 3
    2 3 4
    3 4 2

    1 1 1
    0 1 1
    0 0 0

    self.backward_gauss()


    1 0 0|3
    0 1 1|4


    x_1 = 3
    x_2 = 4 - x_3
    x_3

    self.get_solution()

    

class Solution:
    is_empty() -> bool
    num_free_vars() -> int
    get_some_solution(*args) -> Optional[List]:
        len(args) == self.num_free_vars()
        


    

    lambda x: [3, 4-x, x]
    lambda: [3, 4, 1]
    lambda: None

    
    


SyntaxError: unmatched ')' (3085257351.py, line 5)